# Actividad 3.2

In [1]:
import numpy as np
from math import erf, sqrt, comb, floor

## Ejercicio 1

In [2]:
def phi(x):
    """Densidad normal estándar φ(x)."""
    return (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2)

# Parámetros del rectángulo
a, b = -4.0, 1.65                      # intervalo acotado
c = 1/np.sqrt(2*np.pi)                 # c = max φ(x) = φ(0)
n = 300_000                            # número de puntos
rng = np.random.default_rng(7)         # semilla para reproducibilidad

# Muestreo uniforme en el rectángulo [a,b]×[0,c]
x = rng.uniform(a, b, n)
y = rng.uniform(0.0, c, n)

# Conteo de puntos bajo la curva
n0 = np.sum(y <= phi(x))

# Estimador clásico: θ̂ = c (b−a) · (n0/n)
rect_area = c * (b - a)
p_hat = n0 / n
theta_hat = rect_area * p_hat

# Error estándar plug-in usando Var(θ̂) = θ/n [ c(b−a) − θ ] (desconocido ⇒ sustituimos θ̂)
var_hat = (theta_hat / n) * (rect_area - theta_hat)
se_hat = np.sqrt(var_hat)

# Valor "teórico" con CDF normal vía erf (para comparar)
def Phi(z):  # CDF normal estándar
    return 0.5*(1 + erf(z/sqrt(2)))

true_full = Phi(1.65)           # ∫_{-∞}^{1.65} φ
left_tail = Phi(-4.0)           # ∫_{-∞}^{-4} φ (muy pequeño)
true_trunc = true_full - left_tail   # ∫_{-4}^{1.65} φ

# IC aproximado al 95%
ci_low = theta_hat - 1.96*se_hat
ci_high = theta_hat + 1.96*se_hat

print(f"Estimación Monte Carlo (truncada [-4,1.65]): {theta_hat:.6f}")
print(f"Error estándar (plug-in): {se_hat:.6f}")
print(f"IC 95% (θ̂ ± 1.96·SE): [{ci_low:.6f}, {ci_high:.6f}]")
print(f"Valor teórico ∫_(-∞)^(1.65) φ = {true_full:.6f}")
print(f"Cola izquierda Φ(-4) = {left_tail:.8f}  ⇒  valor truncado ≈ {true_trunc:.6f}")


Estimación Monte Carlo (truncada [-4,1.65]): 0.953715
Error estándar (plug-in): 0.002033
IC 95% (θ̂ ± 1.96·SE): [0.949730, 0.957700]
Valor teórico ∫_(-∞)^(1.65) φ = 0.950529
Cola izquierda Φ(-4) = 0.00003167  ⇒  valor truncado ≈ 0.950497


## Ejercicio 2

In [3]:
def mc_clasico_hit_or_miss(g, a, b, c, n=200_000, seed=123):
    """
    Método Monte Carlo clásico (hit-or-miss) para ∫_a^b g(x) dx con 0 <= g(x) <= c.
    Devuelve (theta_hat, se_hat, n0, rect_area).
    """
    rng = np.random.default_rng(seed)
    x = rng.uniform(a, b, size=n)
    y = rng.uniform(0.0, c, size=n)
    under = y <= g(x)
    n0 = int(np.sum(under))
    rect_area = c * (b - a)
    p_hat = n0 / n
    theta_hat = rect_area * p_hat
    # Var(θ̂) ≈ θ̂/n * (c(b−a) − θ̂)  (plug-in)
    var_hat = (theta_hat / n) * (rect_area - theta_hat)
    se_hat = np.sqrt(max(var_hat, 0.0))
    return theta_hat, se_hat, n0, rect_area

# (a) g(x)=c0 constante
a, b, c0 = 0.0, 3.0, 5.0
g_const = lambda x: c0*np.ones_like(x)
theta_hat_a, se_hat_a, _, _ = mc_clasico_hit_or_miss(g_const, a, b, c=c0, n=100_000, seed=7)
print("(a) g(x)=c0 en (a,b):   θ̂ =", theta_hat_a, "  SE ≈", se_hat_a, "  (valor real =", c0*(b-a), ")")

# (b) g(x)=2x en (0,1) — caso correcto: c=2
g_lin = lambda x: 2.0*x
theta_hat_b_ok, se_hat_b_ok, _, _ = mc_clasico_hit_or_miss(g_lin, 0.0, 1.0, c=2.0, n=200_000, seed=7)
print("(b) g(x)=2x, c=2:       θ̂ ≈", round(theta_hat_b_ok,6), "  SE ≈", round(se_hat_b_ok,6), "  (valor real = 1)")

# (b) Si alguien usa c=1 (no válido): muestra el sesgo hacia 3/4
theta_hat_b_bad, se_hat_b_bad, _, _ = mc_clasico_hit_or_miss(g_lin, 0.0, 1.0, c=1.0, n=200_000, seed=7)
print("(b) g(x)=2x, c=1 (!)    θ̂ ≈", round(theta_hat_b_bad,6), "  SE ≈", round(se_hat_b_bad,6), "  (valor 'truncado' = 0.75)")


(a) g(x)=c0 en (a,b):   θ̂ = 15.0   SE ≈ 0.0   (valor real = 15.0 )
(b) g(x)=2x, c=2:       θ̂ ≈ 1.00249   SE ≈ 0.002236   (valor real = 1)
(b) g(x)=2x, c=1 (!)    θ̂ ≈ 0.75134   SE ≈ 0.000967   (valor 'truncado' = 0.75)


## Ejercicio 3

In [4]:
def volumen_esfera_mc(r=1.0, n=1_000_000, seed=42):
    """
    Volumen de una esfera de radio r mediante Monte Carlo clásico (hit-or-miss) en 3D.
    Idea: muestrear uniforme en el cubo [-r, r]^3 y contar la fracción de puntos
    que cae dentro de la esfera x^2 + y^2 + z^2 <= r^2.
    """
    rng = np.random.default_rng(seed)

    # 1) Volumen del cubo contenedor: V_cubo = (2r)^3
    a, b = -r, r
    V_cubo = (b - a) ** 3

    # 2) Muestreo uniforme en el cubo
    x = rng.uniform(a, b, size=n)
    y = rng.uniform(a, b, size=n)
    z = rng.uniform(a, b, size=n)

    # 3) Indicador: ¿cayó dentro de la esfera?
    inside = (x*x + y*y + z*z) <= (r*r)
    n0 = int(np.sum(inside))

    # 4) Estimador clásico: V̂ = V_cubo * (n0/n)
    p_hat = n0 / n
    V_hat = V_cubo * p_hat

    # 5) Error estándar (plug-in) por binomial: Var(V̂) = V_cubo^2 * p(1-p)/n
    #    Forma equivalente: Var(V̂) = (V_hat/n) * (V_cubo - V_hat)
    var_hat = (V_hat / n) * (V_cubo - V_hat)
    se_hat = np.sqrt(max(var_hat, 0.0))

    # 6) Valor exacto y un IC 95% (approx)
    V_true = (4.0/3.0) * np.pi * (r**3)
    ci = (V_hat - 1.96*se_hat, V_hat + 1.96*se_hat)

    return {
        "V_hat": V_hat,
        "SE": se_hat,
        "n": n,
        "r": r,
        "V_cubo": V_cubo,
        "n0": n0,
        "p_hat": p_hat,
        "IC95": ci,
        "V_true": V_true,
    }

# Ejemplo de uso:
res = volumen_esfera_mc(r=1.0, n=1_000_000, seed=7)
print(f"Volumen estimado = {res['V_hat']:.5f}")
print(f"Error estándar   = {res['SE']:.6f}")
print(f"IC 95%           = [{res['IC95'][0]:.5f}, {res['IC95'][1]:.5f}]")
print(f"Volumen exacto   = {res['V_true']:.5f}")


Volumen estimado = 4.19170
Error estándar   = 0.003995
IC 95%           = [4.18387, 4.19953]
Volumen exacto   = 4.18879


## Ejercicio 4

In [5]:
def area_region_hit_or_miss(n=300_000, seed=123, tight=True):
    """
    Estima el área de {(x,y): -1<x<1, y>0, sqrt(1-2x^2) < y < sqrt(1-2x^4)}
    por 'hit-or-miss'.

    Si tight=True usa el rectángulo [-1/sqrt(2), 1/sqrt(2)] x [0, 1],
    que contiene a la región y reduce varianza.
    """
    rng = np.random.default_rng(seed)
    if tight:
        a, b = -1/np.sqrt(2), 1/np.sqrt(2)
    else:
        a, b = -1.0, 1.0
    c, d = 0.0, 1.0                      # en y basta [0,1], pues y<sqrt(1-2x^4)≤1
    area_rect = (b - a) * (d - c)

    # Muestras uniformes en el rectángulo
    x = rng.uniform(a, b, size=n)
    y = rng.uniform(c, d, size=n)

    # Radicandos (para evitar sqrt de negativos)
    r1 = 1 - 2*x**2
    r2 = 1 - 2*x**4
    valid = (r1 >= 0) & (r2 >= 0)

    y_low  = np.zeros_like(x)
    y_high = np.zeros_like(x)
    y_low[valid]  = np.sqrt(r1[valid])
    y_high[valid] = np.sqrt(r2[valid])

    inside = valid & (y > y_low) & (y < y_high)
    p_hat = np.mean(inside)
    area_hat = area_rect * p_hat

    # SE (plug-in) para hit-or-miss: Var = A/n * (AreaRect - A)
    var_hat = (area_hat / n) * (area_rect - area_hat)
    se_hat = np.sqrt(max(var_hat, 0.0))
    ci95 = (area_hat - 1.96*se_hat, area_hat + 1.96*se_hat)

    return {"area_hat": area_hat, "SE": se_hat, "IC95": ci95,
            "n": n, "rect_area": area_rect, "p_hat": p_hat}

# Ejemplo rápido:
res = area_region_hit_or_miss(n=400_000, seed=7, tight=True)
res


{'area_hat': np.float64(0.2264580177428037),
 'SE': np.float64(0.0008200255578334128),
 'IC95': (np.float64(0.2248507676494502), np.float64(0.2280652678361572)),
 'n': 400000,
 'rect_area': np.float64(1.414213562373095),
 'p_hat': np.float64(0.16013)}

## Ejercicio 5

In [6]:
# -------------------------------
# 1) Simulación base: S = X1+X2+X3 con Xi ~ Unif(0,15)
# -------------------------------
def simular_sumas(n=1_000_000, seed=123):
    rng = np.random.default_rng(seed)
    X = rng.uniform(0.0, 15.0, size=(n, 3))
    S = X.sum(axis=1)
    S.sort()  # ordenar una vez -> ECDF eficiente/estable
    return S

# -------------------------------
# 2) ECDF y probabilidad estimada P(S <= x)
# -------------------------------
def p_hat_ecdf(S_sorted, x):
    """
    Devuelve hat{p}(x) = (1/n) * #{ S_i <= x } usando búsqueda binaria
    sobre el arreglo ordenado S_sorted.
    """
    import bisect
    n = len(S_sorted)
    idx = bisect.bisect_right(S_sorted, x)
    return idx / n

# -------------------------------
# 3) Buscar x con p_hat(x) ≈ objetivo (búsqueda dicotómica sobre ECDF)
# -------------------------------
def buscar_x_para_prob(S_sorted, objetivo=0.95, lo=0.0, hi=45.0, tol=1e-4, maxit=60):
    p_lo = p_hat_ecdf(S_sorted, lo)
    p_hi = p_hat_ecdf(S_sorted, hi)
    if objetivo <= p_lo:
        return lo
    if objetivo >= p_hi:
        return hi

    for _ in range(maxit):
        mid = 0.5 * (lo + hi)
        p_mid = p_hat_ecdf(S_sorted, mid)
        if abs(p_mid - objetivo) <= tol:
            return mid
        if p_mid < objetivo:
            lo = mid
        else:
            hi = mid
    return 0.5 * (lo + hi)

# -------------------------------
# 4) Cuantil empírico (ECDF): alternativa directa
# -------------------------------
def cuantil_empirico(S_sorted, q=0.95):
    """
    Devuelve el cuantil empírico de orden q de S_sorted (arreglo ordenado).
    """
    n = len(S_sorted)
    # índice "linear" equivalente a np.quantile con método 'linear'
    pos = q * (n - 1)
    i = int(np.floor(pos))
    frac = pos - i
    if i + 1 < n:
        return (1 - frac) * S_sorted[i] + frac * S_sorted[i + 1]
    else:
        return S_sorted[-1]

# -------------------------------
# Ejecución de ejemplo
# -------------------------------
if __name__ == "__main__":
    # Generar una sola muestra grande -> se usa para todos los x (comparaciones estables)
    S = simular_sumas(n=1_000_000, seed=7)

    # Aproximar x con P(S<=x)=0.95 probando distintos x vía búsqueda dicotómica
    x95_biseccion = buscar_x_para_prob(S, objetivo=0.95)

    # Alternativa: cuantil empírico 0.95 directamente
    x95_empirico = cuantil_empirico(S, q=0.95)

    # Verificación rápida con normal aproximada
    mean_S = 3 * 15 / 2         # = 22.5
    sd_S   = np.sqrt(3 * (15**2) / 12)  # = 7.5
    x95_normal = mean_S + 1.645 * sd_S


    print(f"x (búsqueda sobre ECDF)  ≈ {x95_biseccion:.6f}")
    print(f"x (cuantil empírico 0.95)≈ {x95_empirico:.6f}")
    print(f"check normal aprox       ≈ {x95_normal:.6f}")


x (búsqueda sobre ECDF)  ≈ 34.958496
x (cuantil empírico 0.95)≈ 34.954878
check normal aprox       ≈ 34.837500


## Ejercicio 6

In [7]:
# ---------- Analítico ----------
def normal_cdf(z: float) -> float:
    """Φ(z) usando erf (sin dependencias externas)."""
    return 0.5 * (1.0 + erf(z / sqrt(2.0)))

def prob_analitica():
    """
    P(X10 > 12) con X_t = 3*sqrt(t) + B_t,  B_t ~ N(0.2 t, 0.32 t), t=10.
    Devuelve (p, mu, sigma, z0).
    """
    t = 10.0
    mu = 3.0*sqrt(t) + 0.2*t          # = 3*sqrt(10) + 2
    sigma = sqrt(0.32*t)              # = sqrt(3.2) = 4/sqrt(5)
    z0 = (12.0 - mu) / sigma
    p = 1.0 - normal_cdf(z0)
    return p, mu, sigma, z0

# ---------- Simulación (media muestral) ----------
def prob_mc(n=500_000, seed=7):
    """
    Estima P(X10 > 12) por simulación.
    """
    t = 10.0
    mu_B, var_B = 0.2*t, 0.32*t       # B10 ~ N(2, 3.2)
    sd_B = sqrt(var_B)
    drift = 3.0*sqrt(t)               # 3*sqrt(10)

    rng = np.random.default_rng(seed)
    Z = rng.standard_normal(n)
    B10 = mu_B + sd_B * Z
    X10 = drift + B10

    I = (X10 > 12.0)
    p_hat = float(np.mean(I))
    se = float(np.sqrt(p_hat * (1.0 - p_hat) / n))
    ci = (p_hat - 1.96*se, p_hat + 1.96*se)
    return p_hat, se, ci

# ---------- Ejecución de ejemplo ----------
if __name__ == "__main__":
    p_ex, mu, sigma, z0 = prob_analitica()
    p_mc, se_mc, ci_mc = prob_mc(n=500_000, seed=7)

    print(f"Analítico:  P(X10>12) = {p_ex:.6f}  "
          f"(mu={mu:.6f}, sigma={sigma:.6f}, z0={(z0):.6f})")
    print(f"Monte Carlo: p_hat={p_mc:.6f}, SE={se_mc:.6f}, "
          f"IC95%=[{ci_mc[0]:.6f}, {ci_mc[1]:.6f}]")


Analítico:  P(X10>12) = 0.387106  (mu=11.486833, sigma=1.788854, z0=0.286869)
Monte Carlo: p_hat=0.387350, SE=0.000689, IC95%=[0.386000, 0.388700]


## Ejercicio 7

In [8]:
def sim_ej7(n=300_000, u=1.0, c=3.0, lam=2.0, t=5.0, seed=7):
    """
    Proceso de riesgo clásico:
      C_t = u + c t - sum_{j=1}^{N_t} Y_j,   N_t ~ Poisson(lam * t),  Y_j ~ Exp(1).

    Devuelve un dict con:
      - p_ruin:           estimación de P(C5 < 0)
      - se_ruin:          error estándar (binomial) de p_ruin
      - p_cond_uncond:    estimación de P(C5 < 0 | Y1 < 1)  (condición 'solo Y1<1')
      - se_cond_uncond:   su error estándar sobre m = # {Y1<1}
      - p_cond_atleast1:  estimación de P(C5 < 0 | N5>=1, Y1 < 1)  (lectura alternativa)
      - se_cond_atleast1: su error estándar sobre m = # {N5>=1, Y1<1}
      - mean_N:           media muestral de N5 (control de calidad; ~ 10)
    """
    rng = np.random.default_rng(seed)
    premium = u + c * t  # u + c t

    # 1) Número de siniestros
    N = rng.poisson(lam * t, size=n)   # N5

    # 2) Primer siniestro Y1 (definido siempre; independiente de N)
    Y1 = rng.exponential(scale=1.0, size=n)

    # 3) Suma total S = Y1 + Gamma(N-1,1) si N>=1; S=0 si N=0
    S = np.zeros(n)
    mask_ge1 = (N >= 1)
    mask_ge2 = (N >= 2)
    S[mask_ge1] = Y1[mask_ge1]                 # incluye Y1
    if np.any(mask_ge2):                       # faltan N-1 ≥ 1 reclamaciones
        k_rest = N[mask_ge2] - 1              # parámetros forma (enteros)
        S[mask_ge2] += rng.gamma(shape=k_rest, scale=1.0)

    # 4) Capital al tiempo t
    C = premium - S

    # (a) P(C5 < 0)
    ruin = (C < 0.0)
    p_ruin = float(np.mean(ruin))
    se_ruin = float(np.sqrt(p_ruin * (1.0 - p_ruin) / n))
    ic95_ruin = (p_ruin - 1.96*se_ruin, p_ruin + 1.96*se_ruin)

    # (b1) Condición "solo Y1<1"
    den1 = (Y1 < 1.0)
    m1 = int(np.count_nonzero(den1))
    p_cond_uncond = float(np.mean(ruin[den1])) if m1 > 0 else np.nan
    se_cond_uncond = float(np.sqrt(p_cond_uncond * (1.0 - p_cond_uncond) / m1)) if m1 > 0 else np.nan
    ic95_cu = (p_cond_uncond - 1.96*se_cond_uncond, p_cond_uncond + 1.96*se_cond_uncond) if m1 > 0 else (np.nan, np.nan)

    # (b2) Condición "Y1<1 y N5>=1"
    den2 = den1 & (N >= 1)
    m2 = int(np.count_nonzero(den2))
    p_cond_atleast1 = float(np.mean(ruin[den2])) if m2 > 0 else np.nan
    se_cond_atleast1 = float(np.sqrt(p_cond_atleast1 * (1.0 - p_cond_atleast1) / m2)) if m2 > 0 else np.nan
    ic95_ca = (p_cond_atleast1 - 1.96*se_cond_atleast1, p_cond_atleast1 + 1.96*se_cond_atleast1) if m2 > 0 else (np.nan, np.nan)

    return {
        "n": n,
        "premium": premium,
        "mean_N": float(np.mean(N)),
        "p_ruin": p_ruin, "se_ruin": se_ruin, "IC95_ruin": ic95_ruin,
        "p_cond_uncond": p_cond_uncond, "se_cond_uncond": se_cond_uncond, "IC95_cond_uncond": ic95_cu, "m_uncond": m1,
        "p_cond_atleast1": p_cond_atleast1, "se_cond_atleast1": se_cond_atleast1, "IC95_cond_atleast1": ic95_ca, "m_atleast1": m2
    }

# Ejemplo de uso:
res = sim_ej7(n=300_000, seed=7)
for k, v in res.items():
    print(f"{k}: {v}")


n: 300000
premium: 16.0
mean_N: 9.992766666666666
p_ruin: 0.09878
se_ruin: 0.0005447400346342587
IC95_ruin: (0.09771230953211686, 0.09984769046788315)
p_cond_uncond: 0.07879677337612974
se_cond_uncond: 0.0006190339111531578
IC95_cond_uncond: (0.07758346691026954, 0.08001007984198993)
m_uncond: 189424
p_cond_atleast1: 0.07880301358435977
se_cond_atleast1: 0.0006190808379106504
IC95_cond_atleast1: (0.0775896151420549, 0.08001641202666465)
m_atleast1: 189409


## Ejercicio 9

In [9]:
# ---------------------------
# Probabilidades exactas
# ---------------------------
def exact_a(): return 0.5
def exact_b(): return np.pi / 4.0
def exact_c(): return 1.0

# ---------------------------
# p(x) condicionales
# ---------------------------
def p_a_of_x(x):
    # piecewise: 0, (2x+1)/2, 1
    out = np.zeros_like(x, dtype=float)
    mask_mid = (x > -0.5) & (x < 0.5)
    mask_hi  = (x >= 0.5)
    out[mask_mid] = (2.0 * x[mask_mid] + 1.0) / 2.0
    out[mask_hi]  = 1.0
    return out

def p_b_of_x(x):
    # sqrt(1 - x^2), safe
    val = 1.0 - x*x
    val[val < 0] = 0.0
    return np.sqrt(val)

def p_c_of_x(x):
    # always 1 on (-1,1)
    return np.ones_like(x, dtype=float)

# ---------------------------
# Estimadores por muestreo condicional
# ---------------------------
def cond_estimator(p_of_x, n=200_000, seed=7):
    rng = np.random.default_rng(seed)
    x = rng.uniform(-1.0, 1.0, size=n)      # X ~ Unif(-1,1)
    p_vals = p_of_x(x)                      # p(X)
    theta_hat = float(np.mean(p_vals)) / 1.0  # E[p(X)] (f_X=1/2 se refleja en p(x) ya definido? -> p(x) arriba ya es prob condicional; su promedio directo es la prob, porque usamos media muestral sobre X ~ Unif(-1,1) y la densidad 1/2 se incorpora en la esperanza -> tomar mean(p_vals) está bien si x ~ Unif(-1,1) con peso uniforme; estrictamente E[p(X)] = ∫ p(x) f_X dx; discretamente es el promedio simple al muestrear de f_X)
    # Error estándar (de la media de p(X))
    se = float(np.std(p_vals, ddof=1) / np.sqrt(n))
    ci = (theta_hat - 1.96 * se, theta_hat + 1.96 * se)
    return theta_hat, se, ci

# ---------------------------
# Estimadores “crudos” para comparar (opcional)
# ---------------------------
def crude_estimator(event_fn, n=200_000, seed=7):
    rng = np.random.default_rng(seed)
    x = rng.uniform(-1.0, 1.0, size=n)
    y = rng.uniform(-1.0, 1.0, size=n)
    I = event_fn(x, y).astype(float)
    p = float(np.mean(I))
    se = float(np.sqrt(p * (1 - p) / n))
    ci = (p - 1.96 * se, p + 1.96 * se)
    return p, se, ci

def event_a(x, y): return (4.0*x - 2.0*y) > 0.0
def event_b(x, y): return (x*x + y*y) < 1.0
def event_c(x, y): return (x*x/3.0 + y*y/2.0) < 1.0

# ---------------------------
# Ejemplo de uso
# ---------------------------
if __name__ == "__main__":
    n = 200_000

    # (a)
    th_a, se_a, ci_a = cond_estimator(p_a_of_x, n=n, seed=1)
    cr_a, secr_a, ccr_a = crude_estimator(event_a, n=n, seed=1)
    print("(a) cond:", th_a, "SE:", se_a, "IC95:", ci_a, "| exacto:", exact_a())
    print("    crudo:", cr_a, "SE:", secr_a, "IC95:", ccr_a)

    # (b)
    th_b, se_b, ci_b = cond_estimator(p_b_of_x, n=n, seed=2)
    cr_b, secr_b, ccr_b = crude_estimator(event_b, n=n, seed=2)
    print("(b) cond:", th_b, "SE:", se_b, "IC95:", ci_b, "| exacto:", exact_b())
    print("    crudo:", cr_b, "SE:", secr_b, "IC95:", ccr_b)

    # (c)
    th_c, se_c, ci_c = cond_estimator(p_c_of_x, n=n, seed=3)
    cr_c, secr_c, ccr_c = crude_estimator(event_c, n=n, seed=3)
    print("(c) cond:", th_c, "SE:", se_c, "IC95:", ci_c, "| exacto:", exact_c())
    print("    crudo:", cr_c, "SE:", secr_c, "IC95:", ccr_c)


(a) cond: 0.49944514018736547 SE: 0.0009129076713279941 IC95: (0.4976558411515626, 0.5012344392231683) | exacto: 0.5
    crudo: 0.49887 SE: 0.0011180311335110486 IC95: (0.4966786589783183, 0.5010613410216816)
(b) cond: 0.7847524382953979 SE: 0.0004990404544118926 IC95: (0.7837743190047506, 0.7857305575860452) | exacto: 0.7853981633974483
    crudo: 0.785135 SE: 0.0009184172030591543 IC95: (0.7833349022820041, 0.786935097717996)
(c) cond: 1.0 SE: 0.0 IC95: (1.0, 1.0) | exacto: 1.0
    crudo: 1.0 SE: 0.0 IC95: (1.0, 1.0)


## Ejercicio 10

In [10]:
pN1, pN2 = 3/5, 2/5
E_N = 1*pN1 + 2*pN2                 # = 7/5
# E[X] por series: (3/10) * [ sum x (1/2)^x + sum x (1/4)^x ]
E_X = (3/10) * ((0.5) / (1-0.5)**2 + (0.25) / (1-0.25)**2)  # = 11/15
E_S_exact = E_N * E_X                                   # = 77/75

def sample_N(rng, size):
    """Muestra N in {1,2} con P(N=1)=3/5, P(N=2)=2/5."""
    u = rng.random(size)
    return np.where(u < pN1, 1, 2)

def sample_X_given_n(rng, n):
    """
    Muestra X | N=n. Condicional es geométrica en {0,1,...} con p_n=1-(1/2)^n.
    Numpy geometric devuelve {1,2,...} con media 1/p -> restamos 1.
    """
    pn = 1 - 0.5**n
    return rng.geometric(p=pn) - 1

def sample_X_marginal(rng, size):
    """
    Muestra X de su marginal P(X=x) = (3/10)[2^{-x} + 4^{-x}].
    Se usa el truco de mezcla: elegir N' ~ {1,2} con (3/5, 2/5) y luego X|N'.
    """
    Nprime = sample_N(rng, size)
    return sample_X_given_n(rng, Nprime)

# ---------- Simulación condicional sobre N ----------
def estimate_ES_conditional(m=200_000, seed=7):
    rng = np.random.default_rng(seed)
    N = sample_N(rng, m)
    mu_X = E_X                        # usar el valor exacto (puedes reemplazar por estimado si quieres)
    S_hat = N * mu_X
    mu_hat = float(np.mean(S_hat))
    # Error estándar de la media de N*mu_X:
    se = float(np.std(S_hat, ddof=1) / np.sqrt(m))
    ci = (mu_hat - 1.96*se, mu_hat + 1.96*se)
    return mu_hat, se, ci

# ---------- Simulación "cruda" (para comparar) ----------
def estimate_ES_crude(m=200_000, seed=8):
    rng = np.random.default_rng(seed)
    N = sample_N(rng, m)
    S = np.zeros(m, dtype=float)
    # casos N=1 y N=2 (únicos posibles)
    mask1 = (N == 1)
    mask2 = ~mask1
    S[mask1] = sample_X_marginal(rng, np.sum(mask1))
    if np.any(mask2):
        X1 = sample_X_marginal(rng, np.sum(mask2))
        X2 = sample_X_marginal(rng, np.sum(mask2))
        S[mask2] = X1 + X2
    mu_hat = float(np.mean(S))
    se = float(np.std(S, ddof=1) / np.sqrt(m))
    ci = (mu_hat - 1.96*se, mu_hat + 1.96*se)
    return mu_hat, se, ci

# ---------- Demostración ----------
if __name__ == "__main__":
    m = 300_000
    mu_c, se_c, ci_c = estimate_ES_conditional(m=m, seed=7)
    mu_r, se_r, ci_r = estimate_ES_crude(m=m, seed=8)

    print(f"E[S] exacta             = {E_S_exact:.8f}  (= 77/75)")
    print(f"Condicional sobre N     = {mu_c:.8f}   SE={se_c:.6f}  IC95={ci_c}")
    print(f"Crudo (simular X_i)     = {mu_r:.8f}   SE={se_r:.6f}  IC95={ci_r}")
    if se_c > 0:
        print(f"Reducción de varianza ≈ {(se_r**2)/(se_c**2):.2f}x (SE_crudo^2 / SE_cond^2)")


E[S] exacta             = 1.02666667  (= 77/75)
Condicional sobre N     = 1.02729489   SE=0.000656  IC95=(1.0260088398921172, 1.02858093788566)
Crudo (simular X_i)     = 1.03039000   SE=0.002725  IC95=(1.0250482811313377, 1.035731718868662)
Reducción de varianza ≈ 17.25x (SE_crudo^2 / SE_cond^2)


## Ejercicio 11

In [11]:
def exact_E_S():
    # (H_11 - 1)/10
    H11 = sum(1.0/k for k in range(1, 12))
    return (H11 - 1.0) / 10.0

def E_S_cmc(m=100_000, seed=7):
    rng = np.random.default_rng(seed)
    N = rng.integers(1, 11, size=m)          # Unif{1,...,10}
    vals = 1.0 / (N + 1.0)                   # E[X^N | N]
    mu = float(np.mean(vals))
    se = float(np.std(vals, ddof=1) / np.sqrt(m))
    ci = (mu - 1.96*se, mu + 1.96*se)
    return mu, se, ci

def E_S_crudo(m=100_000, seed=8):
    rng = np.random.default_rng(seed)
    N = rng.integers(1, 11, size=m)
    X = rng.uniform(0.0, 1.0, size=m)
    S = X ** N
    mu = float(np.mean(S))
    se = float(np.std(S, ddof=1) / np.sqrt(m))
    ci = (mu - 1.96*se, mu + 1.96*se)
    return mu, se, ci

if __name__ == "__main__":
    exact = exact_E_S()
    mu_cmc, se_cmc, ci_cmc = E_S_cmc(m=200_000, seed=7)
    mu_raw, se_raw, ci_raw = E_S_crudo(m=200_000, seed=7)

    print(f"E[S] exacta      = {exact:.10f}")
    print(f"CMC (solo N)     = {mu_cmc:.10f}  SE={se_cmc:.6f}  IC95={ci_cmc}")
    print(f"Crudo (X^N)      = {mu_raw:.10f}  SE={se_raw:.6f}  IC95={ci_raw}")
    if se_cmc > 0:
        print(f"Reducción var. ≈ {(se_raw**2)/(se_cmc**2):.2f}x (SE_crudo^2/SE_CMC^2)")


E[S] exacta      = 0.2019877345
CMC (solo N)     = 0.2017700258  SE=0.000273  IC95=(0.2012342275226516, 0.20230582406464995)
Crudo (X^N)      = 0.2015832854  SE=0.000620  IC95=(0.20036824333333011, 0.2027983275502032)
Reducción var. ≈ 5.14x (SE_crudo^2/SE_CMC^2)
